In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import os
from datetime import datetime

import numpy as np
import pandas as pd
import random

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC, LinearSVR
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier

from argparse import ArgumentParser

# In[2]:


import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.rdchem import HybridizationType
from rdkit.Chem.rdchem import BondType as BT
from rdkit import RDLogger                                                                                                                                                               
RDLogger.DisableLog('rdApp.*')  

### generate fixed representations 
from rdkit import DataStructs
from rdkit.Chem import MACCSkeys
from descriptastorus.descriptors import rdDescriptors, rdNormalizedDescriptors
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors
import warnings
warnings.filterwarnings('ignore')

In [2]:
mol_rep = 'morganBits' 

In [3]:
### form fixed representations (TODO: save all fixed reps in advance?) 
MORGAN_RADIUS = 2
MORGAN_NUM_BITS = 2048

def generate_reps(smiles, mol_rep: str):
    """Generate fixed molecular representations 
    Inputs:
        smiles: a molecule in SMILES representation 
        mol_rep: representation name - options include morganBits, morganCounts, maccs, physchem, rdkit2d, atomPairs
    """
    mol = Chem.MolFromSmiles(smiles)

    if mol_rep == 'morganBits':
        features_vec = AllChem.GetMorganFingerprintAsBitVect(mol, radius=MORGAN_RADIUS, nBits=MORGAN_NUM_BITS)
        features = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(features_vec, features)
    elif mol_rep == 'morganCounts':
        features_vec = AllChem.GetHashedMorganFingerprint(mol, radius=MORGAN_RADIUS, nBits=MORGAN_NUM_BITS)
        features = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(features_vec, features)
    elif mol_rep == 'maccs':
        features_vec = MACCSkeys.GenMACCSKeys(mol)
        features = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(features_vec, features)
    elif mol_rep == 'physchem':
        # calculate physchem descriptors values 
        # Reference: https://github.com/molML/MoleculeACE/blob/main/MoleculeACE/benchmark/featurization.py
        weight = Descriptors.ExactMolWt(mol)
        logp = Descriptors.MolLogP(mol)
        h_bond_donor = Descriptors.NumHDonors(mol)
        h_bond_acceptors = Descriptors.NumHAcceptors(mol)
        rotatable_bonds = Descriptors.NumRotatableBonds(mol)
        atoms = Chem.rdchem.Mol.GetNumAtoms(mol)
        heavy_atoms = Chem.rdchem.Mol.GetNumHeavyAtoms(mol)
        molar_refractivity = Chem.Crippen.MolMR(mol)
        topological_polar_surface_area = Chem.QED.properties(mol).PSA
        formal_charge = Chem.rdmolops.GetFormalCharge(mol)
        rings = Chem.rdMolDescriptors.CalcNumRings(mol)
        # form features matrix
        features = np.array([weight, logp, h_bond_donor, h_bond_acceptors, rotatable_bonds, atoms, heavy_atoms, molar_refractivity, topological_polar_surface_area, formal_charge, rings])
    elif mol_rep == 'rdkit2d':
        # instantiate a descriptors generator
        generator = rdNormalizedDescriptors.RDKit2DNormalized()
        features = generator.process(smiles)[1:]
        features = [0 if np.isnan(x) else float(x) for x in features]
    elif mol_rep == 'atomPairs':
        features_vec = rdMolDescriptors.GetHashedAtomPairFingerprint(mol, nBits=2048, use2D=True)
        features = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(features_vec, features)
    else:
        raise ValueError('Not defined fingerprint!')

    return features

In [4]:
# ## load data

smiles_tasks_df = pd.read_csv('./logPapp_external_set.csv')

In [5]:
fp_colname = [f'Bit{n}' for n in range(MORGAN_NUM_BITS)]

In [6]:
des_colname = [f'Des{n}' for n in range(len(rdNormalizedDescriptors.RDKit2DNormalized().columns))]

In [7]:
fp_df = pd.DataFrame([generate_reps(x, 'morganBits') for x in smiles_tasks_df.smiles], index = smiles_tasks_df.smiles, columns=fp_colname)

In [8]:
rdkit2d_df = pd.DataFrame([generate_reps(x, 'rdkit2d') for x in smiles_tasks_df.smiles], index = smiles_tasks_df.smiles, columns=des_colname)

In [9]:
combined_df = pd.concat([fp_df,rdkit2d_df], axis=1)

In [10]:
## load model
from joblib import load
GB_combined = load('./saved_models/GB_combined.joblib') 
GB_fp = load('./saved_models/GB_fp.joblib') 
GB_rdkit2d = load('./saved_models/GB_rdkit2d.joblib') 

RF_combined = load('./saved_models/RF_combined.joblib') 
RF_fp = load('./saved_models/RF_fp.joblib') 
RF_rdkit2d = load('./saved_models/RF_rdkit2d.joblib') 

SVM_combined = load('./saved_models/SVM_combined.joblib') 
SVM_fp = load('./saved_models/SVM_fp.joblib') 
SVM_rdkit2d = load('./saved_models/SVM_rdkit2d.joblib') 

XGB_combined = load('./saved_models/XGB_combined.joblib') 
XGB_fp = load('./saved_models/XGB_fp.joblib') 
XGB_rdkit2d = load('./saved_models/XGB_rdkit2d.joblib') 

In [11]:
GB_combined_pred = GB_combined.predict(combined_df)
RF_combined_pred = RF_combined.predict(combined_df)
SVM_combined_pred = SVM_combined.predict(combined_df)
XGB_combined_pred = XGB_combined.predict(combined_df)

In [12]:
GB_fp_pred = GB_fp.predict(fp_df)
RF_fp_pred = RF_fp.predict(fp_df)
SVM_fp_pred = SVM_fp.predict(fp_df)
XGB_fp_pred = XGB_fp.predict(fp_df)

In [13]:
GB_rdkit2d_pred = GB_rdkit2d.predict(rdkit2d_df)
RF_rdkit2d_pred = RF_rdkit2d.predict(rdkit2d_df)
SVM_rdkit2d_pred = SVM_rdkit2d.predict(rdkit2d_df)
XGB_rdkit2d_pred = XGB_rdkit2d.predict(rdkit2d_df)

In [14]:
result_dict = {'XGB_fp': XGB_fp_pred,
'SVM_fp': SVM_fp_pred,
'GB_fp': GB_fp_pred,
'RF_fp': RF_fp_pred,
'XGB_rdkit2d': XGB_rdkit2d_pred,
'SVM_rdkit2d': SVM_rdkit2d_pred,
'GB_rdkit2d': GB_rdkit2d_pred,
'RF_rdkit2d': RF_rdkit2d_pred,
'XGB_combined': XGB_combined_pred,
'SVM_combined': SVM_combined_pred,
'GB_combined': GB_combined_pred,
'RF_combined': RF_combined_pred,
'ground_truth': smiles_tasks_df['logPapp']
              }

In [15]:
result = pd.DataFrame(result_dict)

In [16]:
combinednet_pred = pd.read_csv('./logPapp_external_set_pred_combinednet.csv')

In [18]:
dmpnn_pred = pd.read_csv('./logPapp_external_set_pred_dmpnn.csv')

In [19]:
lab2_pred = pd.read_csv('./logPapp_external_set_pred_admetlab2.csv')

In [20]:
lab3_pred = pd.read_csv('./logPapp_external_set_pred_admetlab3.csv')

In [21]:
result['CombinedNet'] = combinednet_pred['logPapp']
result['DMPNN'] = dmpnn_pred['logPapp']
result['ADMETlab2'] = lab2_pred['Caco-2']
result['ADMETlab3'] = lab3_pred['caco2']

In [22]:
result

,XGB_fp,SVM_fp,GB_fp,RF_fp,XGB_rdkit2d,SVM_rdkit2d,GB_rdkit2d,RF_rdkit2d,XGB_combined,SVM_combined,GB_combined,RF_combined,ground_truth,CombinedNet,DMPNN,ADMETlab2,ADMETlab3
0,-5.643911,-5.961470,-5.483297,-5.538459,-6.046221,-5.898155,-6.283740,-5.811782,-6.051782,-6.142560,-5.732677,-5.881886,-4.452225,-5.658178,-6.070380,-5.484,-5.246999
1,-4.886450,-5.075859,-4.960182,-5.077284,-5.079074,-5.023228,-4.783831,-5.026886,-5.060615,-4.800673,-5.177343,-4.970772,-4.040959,-4.853692,-4.805622,-5.051,-4.981073
2,-4.976178,-5.074016,-4.835692,-5.037226,-4.904591,-5.303246,-4.719715,-4.850136,-5.279327,-5.102093,-5.616375,-4.855914,-4.795880,-5.287908,-5.190197,-4.865,-5.153186
3,-4.751775,-4.777076,-4.647408,-4.848243,-4.554111,-5.084939,-4.462949,-4.496145,-4.583272,-4.689909,-4.637891,-4.499963,-4.403403,-4.624857,-5.191446,-4.759,-4.802940
4,-6.015895,-5.779423,-6.181006,-5.652338,-5.540268,-5.542982,-5.384678,-5.646651,-5.809102,-6.185668,-5.298511,-5.611605,-6.221849,-6.113716,-5.984838,-5.558,-5.394623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,-6.300303,-6.354416,-6.092220,-5.641723,-5.458561,-5.226032,-5.532097,-5.725168,-5.698493,-6.082107,-5.892829,-5.671041,-5.721849,-6.125656,-6.000243,-5.774,-5.811849
267,-5.138891,-5.343050,-5.588239,-4.849472,-4.663667,-5.137931,-4.931800,-4.941493,-4.775093,-5.193229,-5.458596,-4.871698,-4.662868,-5.011135,-5.166664,-4.808,-4.833522
268,-5.157030,-5.593155,-4.901261,-5.119306,-5.908757,-5.624952,-5.672353,-5.672681,-5.440757,-5.688264,-5.494064,-5.640029,-4.736830,-5.302046,-5.351270,-5.189,-5.234597
269,-5.235996,-5.057087,-5.142637,-5.245434,-4.784398,-5.308624,-4.934321,-4.917437,-4.742096,-4.811804,-4.811503,-4.859504,-4.700732,-4.978535,-5.206163,-5.194,-5.023351


In [23]:
from scipy.stats import spearmanr, pearsonr
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [24]:
metrics_df = pd.DataFrame(index=['r2_score', 'MAE', 'RMSE','spearmanr','pearsonr'],columns=['XGB_fp',
                                                                                           'SVM_fp',
                                                                                            'GB_fp',
                                                                                            'RF_fp',
                                                                                            'XGB_rdkit2d',
                                                                                            'SVM_rdkit2d',
                                                                                            'GB_rdkit2d',
                                                                                            'RF_rdkit2d',
                                                                                            'XGB_combined',
                                                                                            'SVM_combined',
                                                                                            'GB_combined',
                                                                                            'RF_combined',
                                                                                            'CombinedNet',
                                                                                            'DMPNN',
                                                                                            'ADMETlab 2.0',
                                                                                            'ADMETlab 3.0'])

In [25]:
metrics_functions = {
    'r2_score': r2_score,
    'MAE': mean_absolute_error,
    'RMSE': lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)),
    'spearmanr': spearmanr,
    'pearsonr': pearsonr
}

In [26]:
final_df = result.copy()

In [27]:
for metric_name, metric_function in metrics_functions.items():
    if metric_name == 'spearmanr' or metric_name == 'pearsonr':
        xgb = metric_function(final_df['ground_truth'], final_df['XGB_fp'])[0]
        svm = metric_function(final_df['ground_truth'], final_df['SVM_fp'])[0]
        gb = metric_function(final_df['ground_truth'], final_df['GB_fp'])[0]
        rf = metric_function(final_df['ground_truth'], final_df['RF_fp'])[0]
        xgb_2d = metric_function(final_df['ground_truth'], final_df['XGB_rdkit2d'])[0]
        svm_2d = metric_function(final_df['ground_truth'], final_df['SVM_rdkit2d'])[0]
        gb_2d = metric_function(final_df['ground_truth'], final_df['GB_rdkit2d'])[0]
        rf_2d = metric_function(final_df['ground_truth'], final_df['RF_rdkit2d'])[0]
        xgb_com = metric_function(final_df['ground_truth'], final_df['XGB_combined'])[0]
        svm_com = metric_function(final_df['ground_truth'], final_df['SVM_combined'])[0]
        gb_com = metric_function(final_df['ground_truth'], final_df['GB_combined'])[0]
        rf_com = metric_function(final_df['ground_truth'], final_df['RF_combined'])[0]
        combinednet = metric_function(final_df['ground_truth'], final_df['CombinedNet'])[0]
        dmpnn = metric_function(final_df['ground_truth'], final_df['DMPNN'])[0]
        lab2 = metric_function(final_df['ground_truth'], final_df['ADMETlab2'])[0]
        lab3 = metric_function(final_df['ground_truth'], final_df['ADMETlab3'])[0]
        metrics_df.loc[metric_name, 'XGB_fp',] = xgb
        metrics_df.loc[metric_name, 'SVM_fp',] = svm
        metrics_df.loc[metric_name, 'GB_fp',] = gb
        metrics_df.loc[metric_name, 'RF_fp',] = rf
        metrics_df.loc[metric_name, 'XGB_rdkit2d',] = xgb_2d
        metrics_df.loc[metric_name, 'SVM_rdkit2d',] = svm_2d
        metrics_df.loc[metric_name, 'GB_rdkit2d',] = gb_2d
        metrics_df.loc[metric_name, 'RF_rdkit2d',] = rf_2d
        metrics_df.loc[metric_name, 'XGB_combined',] = xgb_com
        metrics_df.loc[metric_name, 'SVM_combined',] = svm_com
        metrics_df.loc[metric_name, 'GB_combined',] = gb_com
        metrics_df.loc[metric_name, 'RF_combined',] = rf_com
        metrics_df.loc[metric_name, 'CombinedNet',] = combinednet
        metrics_df.loc[metric_name, 'DMPNN',] = dmpnn
        metrics_df.loc[metric_name, 'ADMETlab 2.0',] = lab2
        metrics_df.loc[metric_name, 'ADMETlab 3.0'] = lab3
    else:        
        xgb = metric_function(final_df['ground_truth'], final_df['XGB_fp'])
        svm = metric_function(final_df['ground_truth'], final_df['SVM_fp'])
        gb = metric_function(final_df['ground_truth'], final_df['GB_fp'])
        rf = metric_function(final_df['ground_truth'], final_df['RF_fp'])
        xgb_2d = metric_function(final_df['ground_truth'], final_df['XGB_rdkit2d'])
        svm_2d = metric_function(final_df['ground_truth'], final_df['SVM_rdkit2d'])
        gb_2d = metric_function(final_df['ground_truth'], final_df['GB_rdkit2d'])
        rf_2d = metric_function(final_df['ground_truth'], final_df['RF_rdkit2d'])
        xgb_com = metric_function(final_df['ground_truth'], final_df['XGB_combined'])
        svm_com = metric_function(final_df['ground_truth'], final_df['SVM_combined'])
        gb_com = metric_function(final_df['ground_truth'], final_df['GB_combined'])
        rf_com = metric_function(final_df['ground_truth'], final_df['RF_combined'])
        combinednet = metric_function(final_df['ground_truth'], final_df['CombinedNet'])
        dmpnn = metric_function(final_df['ground_truth'], final_df['DMPNN'])
        lab2 = metric_function(final_df['ground_truth'], final_df['ADMETlab2'])
        lab3 = metric_function(final_df['ground_truth'], final_df['ADMETlab3'])
        metrics_df.loc[metric_name, 'XGB_fp',] = xgb
        metrics_df.loc[metric_name, 'SVM_fp',] = svm
        metrics_df.loc[metric_name, 'GB_fp',] = gb
        metrics_df.loc[metric_name, 'RF_fp',] = rf
        metrics_df.loc[metric_name, 'XGB_rdkit2d',] = xgb_2d
        metrics_df.loc[metric_name, 'SVM_rdkit2d',] = svm_2d
        metrics_df.loc[metric_name, 'GB_rdkit2d',] = gb_2d
        metrics_df.loc[metric_name, 'RF_rdkit2d',] = rf_2d
        metrics_df.loc[metric_name, 'XGB_combined',] = xgb_com
        metrics_df.loc[metric_name, 'SVM_combined',] = svm_com
        metrics_df.loc[metric_name, 'GB_combined',] = gb_com
        metrics_df.loc[metric_name, 'RF_combined',] = rf_com
        metrics_df.loc[metric_name, 'CombinedNet',] = combinednet
        metrics_df.loc[metric_name, 'DMPNN',] = dmpnn
        metrics_df.loc[metric_name, 'ADMETlab 2.0',] = lab2
        metrics_df.loc[metric_name, 'ADMETlab 3.0'] = lab3

In [28]:
metrics_df.T

,r2_score,MAE,RMSE,spearmanr,pearsonr
XGB_fp,0.036543,0.589388,0.75056,0.26906,0.299452
SVM_fp,-0.084891,0.649711,0.796457,0.22423,0.229006
GB_fp,-0.073525,0.614953,0.792274,0.212297,0.234742
RF_fp,0.047326,0.571701,0.746348,0.25927,0.259039
XGB_rdkit2d,0.045959,0.597044,0.746884,0.24948,0.318681
SVM_rdkit2d,0.045496,0.632612,0.747065,0.296748,0.320898
GB_rdkit2d,-0.010655,0.609683,0.768724,0.239053,0.306312
RF_rdkit2d,0.082082,0.594202,0.732607,0.300797,0.333168
XGB_combined,0.071125,0.580784,0.736967,0.316483,0.349899
SVM_combined,-0.010685,0.613777,0.768736,0.302638,0.322607
